In [1]:
from glob import glob
from pathlib import Path
# import ROOT
import pandas as pd
import csv
import matplotlib.pyplot as plt

In [ ]:
absolute_path_only = '/media/droc/Seagate/PHYS-461/6-compton/data/' #This will need to change depending on the path to data
input_files = glob(absolute_path_only + '/*.csv')

angle = []
base_file = []
for file in input_files:
    base = Path(file).stem
    base_file.append(base)
    i = base.find('-')
    angle.append(base[:i])

base_file.sort()
angle.sort()
print(angle)
print(len(angle))
print(base_file)

['010', '010', '020', '020', '030', '030', '040', '040', '050', '050', '060', '060', '070', '070', '080', '080', '090', '090', '100', '100', '110', '110']
22
['010-600s-back_11-11', '010-600s-spec_11-11', '020-1200-back_11-11', '020-1200-spec_11-11', '030-1800s-back_11-12', '030-1800s-spec_11-12', '040-2400s-back_11-12', '040-2400s-spec_11-12', '050-3000s-back_11-12', '050-3000s-spec_11-12', '060-3600s-back_11-11', '060-3600s-spec_11-11', '070-4200s-back_11-11', '070-4200s-spec_11-11', '080-4800s-back_11-12', '080-4800s-spec_11-13', '090-5400s-back_11-15', '090-5400s-spec_11-15', '100-6000s-back_11-14', '100-6000s-spec_11-14', '110-6600s-back_11-13', '110-6600s-spec_11-13']


In [57]:
grouped_list = {}
for item in base_file:
    prefix = item.split('-')[0] 
    if prefix not in grouped_list:
        grouped_list[prefix] = [item]
    else:
        grouped_list[prefix].append(item)

# Convert dictionary values into a list of lists
result = list(grouped_list.values())

print(result)

[['010-600s-back_11-11', '010-600s-spec_11-11'], ['020-1200-back_11-11', '020-1200-spec_11-11'], ['030-1800s-back_11-12', '030-1800s-spec_11-12'], ['040-2400s-back_11-12', '040-2400s-spec_11-12'], ['050-3000s-back_11-12', '050-3000s-spec_11-12'], ['060-3600s-back_11-11', '060-3600s-spec_11-11'], ['070-4200s-back_11-11', '070-4200s-spec_11-11'], ['080-4800s-back_11-12', '080-4800s-spec_11-13'], ['090-5400s-back_11-15', '090-5400s-spec_11-15'], ['100-6000s-back_11-14', '100-6000s-spec_11-14'], ['110-6600s-back_11-13', '110-6600s-spec_11-13']]


In [72]:


def breakdown_vals(file_0,file_1):
    minusback_counts = []

    abs_path0 = absolute_path_only + file_0 + ".csv"
    abs_path1 = absolute_path_only + file_1 + ".csv"

    with open(abs_path0, 'r') as file0:
        reader0 = file0.readlines()
        values0 = []
        newlist0 = []
        i = 26
        while i < 1050:
            values0.append(reader0[i])
            cleaned_list0 = [item.replace('\n', '') for item in values0]
            i += 1

    for val in cleaned_list0:
        # print(val)
        newlist0.append(val.split(","))

    df0 = pd.DataFrame(newlist0, columns=['index', 'energy','counts'])

    x0 = []
    counts0 = []

    for i, j in enumerate(df0['energy'].values):
        x0.append(float(j))

    for i, j in enumerate(df0['counts'].values):
        counts0.append(float(j))


    with open(abs_path1, 'r') as file1:

        reader1 = file1.readlines()
        values1 = []
        newlist1 = []
        i = 26
        while i < 1050:
            values1.append(reader1[i])
            cleaned_list1 = [item.replace('\n', '') for item in values1]
            i += 1

    for val in cleaned_list1:
        # print(val)
        newlist1.append(val.split(","))

    df1 = pd.DataFrame(newlist1, columns=['index', 'energy','counts'])

    x1 = []
    counts1 = []

    for i, j in enumerate(df1['energy'].values):
        x1.append(float(j))

    for i, j in enumerate(df1['counts'].values):
        counts1.append(float(j))

    for i in range(len(counts0)):
        minus = counts1[i] - counts0[i]

        minusback_counts.append(minus)

    return x0, minusback_counts



In [75]:
x , counts = breakdown_vals(result[0][0],result[0][1])
newcounts = []
for c in counts:
    v = abs(c)
    newcounts.append(v)

print(len(newcounts))
print(len(x))

plt.hist(x, bins=len(x), weights=newcounts, color='blue', edgecolor='black')
plt.xlabel('Energy')
plt.ylabel('Counts')
plt.title('Weighted Histogram of Energy with Counts')
plt.show()

1024
1024


OSError: [Errno 5] Input/output error